In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
     |████████████████████████████████| 281.4 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 59.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845507 sha256=a41beede757cc3f336fb3ac564a377eb84af24045f96dcbab75cb48a68bb1c1d
  Stored in directory: /nfs/home/leo0511/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,StructType,IntegerType,StructField
spark = SparkSession.builder.appName('CCBDA_HW3').getOrCreate()
train_data = spark.read.csv("./train_student.csv",header=True)
train_metadata = spark.read.csv("./train_meta.csv",header=True)
test_public = spark.read.csv("./test_public.csv",header=True)
test_private = spark.read.csv("./test_private.csv",header=True)
test_all = spark.read.csv("./test_all.csv",header=True)


In [4]:
from pyspark.sql.functions import format_number, lit
train_data = train_data.withColumn('rating', train_data.rating.cast('float'))
train_metadata = train_metadata.withColumn('rating', train_metadata.rating.cast('float'))
#train_metadata = train_metadata.withColumn('helpful', train_metadata.helpful.cast('float'))

In [5]:
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml import Pipeline
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").setHandleInvalid("keep").fit(train_data) for column in list(set(train_data.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexers)
train_data_ID = pipeline.fit(train_data).transform(train_data)
train_metadata_ID = pipeline.fit(train_metadata).transform(train_metadata)

In [6]:
train_metadata_ID.show()

+--------------+----------+--------------------+--------+--------------------+------+--------------------+--------------------+--------------------+----------+----------+
|          user|      item|        reviewerName| helpful|          reviewText|rating|             summary|      unixReviewTime|          reviewTime|user_index|item_index|
+--------------+----------+--------------------+--------+--------------------+------+--------------------+--------------------+--------------------+----------+----------+
|A2OIMJEGOCTQ87|B008H54GVE|                Lisa|  [0, 0]|"My 3yr old got t...|   4.0|My daughter loves it|          1356825600|         12 30, 2012|    1292.0|    5709.0|
|A38KQZS5M1A8T8|B001NLISDG|            rescue25|  [2, 3]|I thought my son ...|   4.0|My 16-month-old l...|          1322870400|          12 3, 2011|    4940.0|    1849.0|
|A1F5O1USOUOOXI|B007S3S8HO|             John K.|  [0, 0]|I'm going to put ...|   3.0| Lots of small parts|          1355443200|         12 14, 20

In [7]:
train_data_ID.show()

+----------+--------------+------+----------+----------+
|      item|          user|rating|user_index|item_index|
+----------+--------------+------+----------+----------+
|B008H54GVE|A2OIMJEGOCTQ87|   4.0|    1292.0|    5709.0|
|B001NLISDG|A38KQZS5M1A8T8|   4.0|    4940.0|    1849.0|
|B007S3S8HO|A1F5O1USOUOOXI|   3.0|     669.0|     937.0|
|B008G6OOHA|A1FTZ5LLEX7NCM|   4.0|     107.0|   10888.0|
|B00A9JNR8E|A1R2JUOGIYH6HO|   4.0|    1210.0|    1979.0|
|B00000JH3R|A2X7C89I7YRX1O|   5.0|    4764.0|     979.0|
|B000RXPU0U|A24BSKCWXC4M6D|   5.0|    2866.0|    6385.0|
|B00DWXUYN0|A3JX1D26WFEXOS|   5.0|    8104.0|    9620.0|
|B000E9DPVI|A2IWHA1XEYSPD6|   5.0|   11230.0|     420.0|
|B001EB9F3C|A2340917M1HHZ3|   4.0|   10687.0|      51.0|
|B00719LBEW|A3K150BGUET5KI|   5.0|    5104.0|    9243.0|
|B00A850S5E|A2S6QCNNN8AK91|   5.0|     145.0|    1734.0|
|B006O6ETJM| AZIRQNN1YYSPB|   2.0|    1141.0|    5595.0|
|B000S1KGGY|A1OQYD2Y1F02WU|   5.0|    1517.0|    4967.0|
|B000TTNW98|A1WVO00PIPF8RA|   5

In [8]:
def get_mat_sparsity(data):
    # Count the total number of ratings in the dataset
    count_nonzero = data.select("rating").count()

    # Count the number of distinct userIds and distinct movieIds
    total_elements = data.select("user_index").distinct().count() * data.select("item_index").distinct().count()

    # Divide the numerator by the denominator
    sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
    print("The ratings dataframe is ", "%.2f" % sparsity + "% sparse.")
    
get_mat_sparsity(train_data_ID)

The ratings dataframe is  99.96% sparse.


In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ALS model
als = ALS(
         userCol="user_index", 
         itemCol="item_index",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop",
         maxIter=20,
         regParam=0.09,
         rank=25
)

In [10]:
model_1 = als.fit(train_data_ID)
model_1

Py4JJavaError: ignored

In [ ]:
test_public_ID = pipeline.fit(test_public).transform(test_public)
test_public_ID = test_public_ID.withColumn('rating', test_public_ID.rating.cast('float'))
test_public_ID.describe().show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np

def find_best_value(test,model):
  # Import the requisite packages
  evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
  public_pred=model.transform(test)
  # rememver to fix nan in prediction
  value = 1
  public_pred_1=public_pred.na.fill(value=value)
  #print("value: ",value)
  rmse_1=evaluator.evaluate(public_pred_1)
  #print("RMSE 1="+str(rmse_1))

  value = 2
  public_pred_2=public_pred.na.fill(value=value)
  #print("value: ",value)
  rmse_2=evaluator.evaluate(public_pred_2)
  #print("RMSE 2="+str(rmse_2))

  value = 3
  public_pred_3=public_pred.na.fill(value=value)
  #print("value: ",value)
  rmse_3=evaluator.evaluate(public_pred_3)
  #print("RMSE 3="+str(rmse_3))

  value = 4
  public_pred_4=public_pred.na.fill(value=value)
  #print("value: ",value)
  rmse_4=evaluator.evaluate(public_pred_4)
  #print("RMSE 4="+str(rmse_4))

  value = 5
  public_pred_5=public_pred.na.fill(value=value)
  #print("value: ",value)
  rmse_5=evaluator.evaluate(public_pred_5)
  #print("RMSE 5="+str(rmse_5))

  rmses = [rmse_1,rmse_2,rmse_3,rmse_4,rmse_5]
  best_value = np.argmin(rmses) +1
  #print("best RMSE: ",rmses[best_value-1])
  #print("best value: ",best_value)
  #public_pred.show()
  return best_value, rmses[best_value-1]

In [ ]:
best_value_1, best_rmse_1 = find_best_value(test_public_ID,model_1)
print("best value 1 ",best_value_1)
print("best rmse 1 ",best_rmse_1)

In [ ]:
test_all_ID = pipeline.fit(test_all).transform(test_all)
#test_all_ID = test_all_ID.withColumn('rating', test_all_ID.rating.cast('float'))
test_all_ID.describe().show()

In [ ]:
all_pred = model_1.transform(test_all_ID)
all_pred = all_pred.na.fill(value=float(best_value_1))
all_pred.show()

In [ ]:
all_pred.describe().show()

In [ ]:
from pyspark.sql.functions import concat, col, lit

out = all_pred.select(concat(col("user"), lit("_"), col("item")) , all_pred.prediction)
out.printSchema()

In [ ]:
out.describe().show()

In [ ]:
out.orderBy("concat(user, _, item)").show()

In [ ]:
path = f"/content/drive/MyDrive/CCBDA/HW5/1221_value{best_value_1}_iter20.csv"
out.orderBy("concat(user, _, item)").coalesce(1).write.format("csv").mode('overwrite').save(path)

In [ ]:
import glob
text = 'U_I,rating'
with open(glob.glob(path+"/*.csv")[0], 'r+') as f:
  content = f.read()
  f.seek(0,0)
  f.write(text+'\n'+content)